In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-0125")

poet_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are the best poet in the world who writes poetry about programming languages. You write poetry by taking advantage of the strengths of any programming language given to you."),
    ("human", "I want to write {programminglanguage} poetry."),
])

poet_chain = poet_prompt | chat 

reviewer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are the best poetry reviewer in the world who interprets poetry about programming languages. When you receive a poem, you summarize the overall theme of the poem and the merits mentioned in the poem, and finally tell me your evaluation rating for the poem (unit: 0.5, minimum 0.0, maximum 5.0)."),
    ("human","{interpretations}")
])

reviewer_chain = reviewer_prompt | chat

# final_chain = {"interpretations": poet_chain} | reviewer_chain
# final_chain.invoke({
#             "programminglanguage" : "Python",
# })

poetry = poet_chain.invoke({
        "programminglanguage" : "Python",
})


review = reviewer_chain.invoke({
    "interpretations" : poetry

})

poetry, review




AIMessage(content="Theme: The poem celebrates the beauty and elegance of the Python programming language. It highlights the importance of whitespace and indentation in Python coding, as well as the simplicity and power of Python syntax. The poet expresses admiration for Python's versatility in various fields like web development and data science, emphasizing the abundance of libraries available for exploration.\n\nMerits: The poem effectively captures the essence of Python as a programming language known for its readability, versatility, and efficiency. The use of poetic language and imagery to describe coding concepts adds a creative and engaging touch to the ode to Python.\n\nEvaluation: 5.0")